In [1]:
from lxml import etree
from datetime import timedelta, date, datetime
import time
from geopy import distance

In [2]:
def extract_gpx(gpx_path):
    tree = etree.parse(gpx_path)
    root = tree.getroot()
    ns = {'default': 'http://www.topografix.com/GPX/1/1',
        'gpxtpx': 'http://www.garmin.com/xmlschemas/TrackPointExtension/v1'}
    trkpts = root.findall('.//default:trkpt', namespaces=ns)
    data_points = []
        
    for trkpt in trkpts:
            lat = trkpt.get('lat')
            lon = trkpt.get('lon')
            time_elem = trkpt.find('default:time', namespaces=ns)
            ele_elem = trkpt.find('default:ele', namespaces=ns)
            hr_elem = trkpt.find('.//gpxtpx:hr', namespaces=ns)
    
            if time_elem is not None and hr_elem is not None:
                timestamp = time_elem.text
                elevation = ele_elem.text
                heart_rate = hr_elem.text
                data_points.append({
                    'timestamp': timestamp,
                    'latitude': lat,
                    'longitude': lon,
                    'elevation': elevation,
                    'heart_rate': heart_rate
                })

    return data_points

In [3]:
marathon_run = extract_gpx('FFM_Marathon_2024.gpx')

In [12]:
def iterate_workout(data_points):
    prev_timestamp = None
    prev_geo = None
    distance_total = 0
    distance_point = 0
    
    speed_point = 0
    speed_rolling = []
    speed_current = 0

    hr_avg = 0
    
    time_passed = 0
    sleep_time = 0
    
    for data in data_points:
        current_timestamp = datetime.strptime(data['timestamp'], '%Y-%m-%dT%H:%M:%SZ')
        current_geo = (data['latitude'], data['longitude'])
        if prev_timestamp is not None:
            sleep_time = (current_timestamp - prev_timestamp).total_seconds()
            time.sleep(max(sleep_time, 0))

        if prev_geo is not None:
            distance_point = distance.distance(prev_geo, current_geo).km
            if sleep_time == 0:
                speed = 0
            else:
                speed_point = distance_point/(sleep_time/3600)
                if len(speed_rolling) >= 10:
                    speed_rolling = speed_rolling[1:]
                speed_rolling.append(speed_point)
                speed_current = round(sum(speed_rolling)/len(speed_rolling),2)

        time_passed += sleep_time
        distance_total += distance_point
        
        if time_passed == 0:
            speed_avg = 0
            hr_avg = float(data['heart_rate'])
        else:
            speed_avg = distance_total/(time_passed/3600)
            hr_avg = (hr_avg*(time_passed-sleep_time)/time_passed)+(float(data['heart_rate'])*sleep_time/time_passed)

        prev_timestamp = current_timestamp
        prev_geo = current_geo
        
        print(f"Time: {data['timestamp']} / HR: {data['heart_rate']} / Avg. HR {hr_avg} / KM/H: {speed_current} / Time passed: {time_passed} / KM:{round(distance_total, 4)}\
        /Avg.KM/H: {speed_avg} / ")       

In [14]:
iterate_workout(marathon_run)

Time: 2024-10-27T09:13:19Z / HR: 163 / Avg. HR 163.0 / KM/H: 0 / Time passed: 0 / KM:0        /Avg.KM/H: 0 / 
Time: 2024-10-27T09:13:20Z / HR: 163 / Avg. HR 163.0 / KM/H: 8.48 / Time passed: 1.0 / KM:0.0024        /Avg.KM/H: 8.480710469317692 / 
Time: 2024-10-27T09:13:21Z / HR: 163 / Avg. HR 163.0 / KM/H: 8.48 / Time passed: 2.0 / KM:0.0047        /Avg.KM/H: 8.480711184656322 / 
Time: 2024-10-27T09:13:22Z / HR: 163 / Avg. HR 163.0 / KM/H: 8.48 / Time passed: 3.0 / KM:0.0071        /Avg.KM/H: 8.48071189660127 / 
Time: 2024-10-27T09:13:23Z / HR: 163 / Avg. HR 163.0 / KM/H: 8.48 / Time passed: 4.0 / KM:0.0094        /Avg.KM/H: 8.480712608653038 / 
Time: 2024-10-27T09:13:24Z / HR: 163 / Avg. HR 163.0 / KM/H: 8.52 / Time passed: 5.0 / KM:0.0118        /Avg.KM/H: 8.521657977781492 / 
Time: 2024-10-27T09:13:25Z / HR: 163 / Avg. HR 163.0 / KM/H: 8.47 / Time passed: 6.0 / KM:0.0141        /Avg.KM/H: 8.474879988618758 / 
Time: 2024-10-27T09:13:26Z / HR: 163 / Avg. HR 163.0 / KM/H: 9.48 / Time pa

KeyboardInterrupt: 

In [ ]:
geopy

<module 'geopy' from 'C:\\Users\\ABurkhard\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\geopy\\__init__.py'>